# XGB for predictions
This Notebook attempts to create the best possible xgb classifier for the earthquake dataset.  

## Imports

In [1]:
## Uncomment this incase you want to run it on google colab
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%pip install -q optuna category-encoders

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import f1_score

# Hyperparameter Optimization
import optuna

# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, FunctionTransformer
from category_encoders import LeaveOneOutEncoder, TargetEncoder, CatBoostEncoder
from sklearn.base import BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin

from pathlib import Path
import pickle
from os import PathLike
import torch

## Constants

In [10]:
# Replace these with your file paths

BASE_DIR = Path('d:\\', 'ml_competitions', 'gorkha_earthquake')
DATA_DIR = BASE_DIR / 'data' / 'raw'
MODEL_DIR = BASE_DIR / 'models'
SUBMISSION_DIR = BASE_DIR / 'submissions'

TRAINING_FEATURES_PATH = DATA_DIR / "train_values.csv"
TRAINING_LABELS_PATH = DATA_DIR / "train_labels.csv"
TEST_FEATURES_PATH = DATA_DIR / "test_values.csv"
SUBMISSION_FORMAT_PATH = DATA_DIR / "submission_format.csv"

## Data Loading

In [11]:
features_df         = pd.read_csv(TRAINING_FEATURES_PATH,   index_col=0)
labels_df           = pd.read_csv(TRAINING_LABELS_PATH,     index_col=0) - 1
test_features_df    = pd.read_csv(TEST_FEATURES_PATH,       index_col=0)

## Data Preprocessing

In [12]:
# Setup

geo_level_columns = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
numerical_columns = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']
categorical_columns = ['foundation_type', 'ground_floor_type', 'land_surface_condition', 
                       'legal_ownership_status', 'other_floor_type',
                       'plan_configuration', 'position', 'roof_type']

In [13]:

class DREncoder(torch.nn.Module):

    def __init__(self, 
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1428,
                 geo_lv3_size: int=12568) -> None:
        super().__init__()
        self.geo_lv1_embedder = torch.nn.Embedding(geo_lv1_size, 16)
        self.geo_lv2_embedder = torch.nn.Embedding(geo_lv2_size, 128)
        self.geo_lv3_embedder = torch.nn.Embedding(geo_lv3_size, 128) 
        self.compressor = torch.nn.Linear(16+128+128, latent_dim)

    def forward(self, x):
        x_1 = self.geo_lv1_embedder(x[:, 0])
        x_2 = self.geo_lv2_embedder(x[:, 1])
        x_3 = self.geo_lv3_embedder(x[:, 2])
        x = torch.concat((x_1, x_2, x_3), dim=1)
        x = torch.nn.functional.relu(x)
        return self.compressor(x)


class GeoDimensionReduction(BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin):

    def __init__(
            self, 
            path: PathLike,
            latent_dim: int=16, 
            geo_lv1_size: int=31,
            geo_lv2_size: int=1418,
            geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.path = path
        self.model = DREncoder(
            latent_dim, 
            geo_lv1_size,
            geo_lv2_size,
            geo_lv3_size
        )
        self.latent_dim = latent_dim
        self.geo_lv1_size = geo_lv1_size
        self.geo_lv2_size = geo_lv2_size
        self.geo_lv3_size = geo_lv3_size
        self.model.load_state_dict(torch.load(path))

    def fit(self, X: pd.DataFrame, y=None, *args, **kwargs):
        return self

    def transform(self, X: pd.DataFrame, y=None, *args, **kwargs):
        # Convert pd to numpy
        if isinstance(X, pd.DataFrame):
            X = X.values # type: ignore
        # Apply encoder
        self.model.eval()
        X = torch.from_numpy(X).type(torch.long) # type: ignore
        return self.model(X).detach().numpy()
    
class RollUpGeoLv3Encoder(torch.nn.Module):

    def __init__(self, 
                 latent_dim: int=16, 
                 geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.geo_lv3_embedder = torch.nn.Embedding(geo_lv3_size, 128)
        self.compressor = torch.nn.Linear(128, latent_dim)

    def forward(self, x):
        x = self.geo_lv3_embedder(x).squeeze(1)
        x = torch.nn.functional.relu(x)
        return self.compressor(x)


class GeoLv3Rollup(BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin):

    def __init__(
            self, 
            path: PathLike,
            latent_dim: int=16, 
            geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.path = path
        self.model = RollUpGeoLv3Encoder(
            latent_dim, 
            geo_lv3_size
        )
        self.latent_dim = latent_dim
        self.geo_lv3_size = geo_lv3_size
        self.model.load_state_dict(torch.load(path))

    def fit(self, X: pd.DataFrame, y=None, *args, **kwargs):
        return self

    def transform(self, X: pd.DataFrame, y=None, *args, **kwargs):
        # Convert pd to numpy
        if isinstance(X, pd.DataFrame):
            X = X.values # type: ignore
        # Apply encoder
        self.model.eval()
        X = torch.from_numpy(X).type(torch.long) # type: ignore
        return self.model(X).detach().numpy()

In [14]:
# Load All Label Encoders
with open(MODEL_DIR / 'geo-lv-1-label-encoder.pickle', 'rb') as f:
    le1 = pickle.load(f)
with open(MODEL_DIR / 'geo-lv-2-label-encoder.pickle', 'rb') as f:
    le2 = pickle.load(f)
with open(MODEL_DIR / 'geo-lv-3-label-encoder.pickle', 'rb') as f:
    le3 = pickle.load(f)

# Prepare Transformers
geo_lv1_le = FunctionTransformer(
    func=lambda x: np.array(le1.transform(x.values.ravel())).reshape(-1, 1),
    feature_names_out='one-to-one'
)

geo_lv2_le = FunctionTransformer(
    func=lambda x: np.array(le2.transform(x.values.ravel())).reshape(-1, 1), 
    feature_names_out='one-to-one'
)

geo_lv3_le = FunctionTransformer(
    func=lambda x: np.array(le3.transform(x.values.ravel())).reshape(-1, 1), 
    feature_names_out='one-to-one'
)

# Dim Reducer
geo_dim_reduction_preprocessor = ColumnTransformer(
    transformers=[
        ('geo1_le', geo_lv1_le, ['geo_level_1_id']),
        ('geo2_le', geo_lv2_le, ['geo_level_2_id']),
        ('geo3_le', geo_lv3_le, ['geo_level_3_id']),
    ], 
    remainder='drop', 
    verbose_feature_names_out=False
).set_output(transform='pandas')

geo_dim_reduction_pipe = Pipeline([
    ('label_encoder', geo_dim_reduction_preprocessor),
    ('embedder', GeoDimensionReduction(path=MODEL_DIR / 'dim-reduction-32', latent_dim=32)),
])

# Rollup
geo3_rollup_preprocessor = ColumnTransformer(
    transformers=[
        ('geo3_le', geo_lv3_le, ['geo_level_3_id']),
    ], 
    remainder='drop', 
    verbose_feature_names_out=False,
).set_output(transform='pandas')

geo_rollup_pipe = Pipeline([
    ('label_encoder', geo3_rollup_preprocessor),
    ('embedder', GeoLv3Rollup(path=MODEL_DIR / 'geo3-rollup-16')),
])

preprocessor_xgboost = ColumnTransformer(
    transformers=[
        # ('bool', FunctionTransformer(lambda x: np.log(1+x), feature_names_out='one-to-one'), numerical_columns),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_columns),
        ('geo_raw', FunctionTransformer(lambda x: x, feature_names_out='one-to-one'), geo_level_columns),
        ('geo_dim_reduction', geo_dim_reduction_pipe, geo_level_columns),
        ('geo_rollup', geo_rollup_pipe, geo_level_columns),
        ('geos', CatBoostEncoder(cols=geo_level_columns), geo_level_columns),
    ],
    remainder='passthrough'
)
# preprocessor_lgbm.set_output(transform='pandas')

## Evaluate Model

In [ ]:
hyperparams = {
    "objective": "multi:softmax",
    "eval_metric": "auc",
    "tree_method": "gpu_hist",
}

xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor_xgboost),
    ('classifier', XGBClassifier(**hyperparams)),
])

results = cross_val_score(
    xgb_pipeline, 
    features_df, 
    labels_df.to_numpy().squeeze(), 
    cv=StratifiedKFold(n_splits=5), 
    scoring='f1_micro',
    verbose=100
)

print(f'{results.mean():.5f}')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] START .....................................................................
[CV] END ................................ score: (test=0.747) total time=   7.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.3s remaining:    0.0s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.743) total time=   6.7s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   14.1s remaining:    0.0s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.750) total time=   7.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   21.2s remaining:    0.0s
[CV] START .....................................................................
[CV] END ................................ score: (test=0.747) total time=   6.7s
[Parallel(n_jobs=1)]: Done   4 

## Hyperparameter Optimization

### Objective Function

In [ ]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(
        trial: optuna.Trial, 
        X_train: np.ndarray, 
        y_train: np.ndarray, 
        X_valid: np.ndarray, 
        y_valid: np.ndarray) -> float:

    hyperparams = {
        "objective": "multi:softmax",
        "eval_metric": "auc",
        "tree_method": "gpu_hist",
        'nthread': 6,
        'seed': 69,
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 10, 15),
        "gamma": trial.suggest_float("gamma", 0, 20),
        "min_child_weight": trial.suggest_int("min_child_weight", 2, 20),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 0.1),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 0.1),
        "subsample": trial.suggest_float("subsample", 0.5, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.9),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 0.9),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 0.9),
        "grow_policy": trial.suggest_categorical("grow_policy", ['depthwise', 'lossguide']),
        # "booster": trial.suggest_categorical("booster", ['gbtree', 'gblinear', 'dart']),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-auc")
    
    xgb_clf = XGBClassifier(
        **hyperparams, 
        callbacks=[pruning_callback], 
        early_stopping_rounds=50,
    )

    xgb_clf.fit(
        X_train, 
        y_train,  
        eval_set=[(X_valid, y_valid)],
        verbose=False,
    )
    results = f1_score(y_valid, xgb_clf.predict(X_valid), average='micro')
    return float(results)

### Study

In [ ]:
study = optuna.create_study(
    study_name='xgb-DR-32-RU-16-Target-study', 
    storage='sqlite:///xgb-DR-32-RU-16-Target.db', 
    load_if_exists=True,
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), 
)

X_train, X_valid, y_train, y_valid = train_test_split(
    features_df, 
    labels_df.to_numpy().squeeze(), 
    stratify=labels_df.to_numpy().squeeze(),
    random_state=69)

X_train = preprocessor_xgboost.fit_transform(X_train, y_train)
X_valid = preprocessor_xgboost.transform(X_valid)

study.optimize(
    lambda trial: objective(trial, 
                            X_train=X_train, 
                            y_train=y_train, 
                            X_valid=X_valid, 
                            y_valid=y_valid), 
    n_trials=100)

[I 2023-05-09 01:31:25,303] A new study created in RDB with name: xgb-DR-32-RU-16-Target-study
[I 2023-05-09 01:31:35,966] Trial 0 finished with value: 0.7299964697395281 and parameters: {'n_estimators': 127, 'learning_rate': 0.00208683675346516, 'max_depth': 11, 'gamma': 16.626939567505225, 'min_child_weight': 4, 'reg_alpha': 0.09915239644768807, 'reg_lambda': 0.0567612348515774, 'subsample': 0.6480905054426477, 'colsample_bytree': 0.7119789021623592, 'colsample_bylevel': 0.678011871799658, 'colsample_bynode': 0.5651564055440244, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.7299964697395281.
[I 2023-05-09 01:31:56,421] Trial 1 finished with value: 0.7313011312182468 and parameters: {'n_estimators': 472, 'learning_rate': 4.294604231789147e-08, 'max_depth': 15, 'gamma': 12.599595316817219, 'min_child_weight': 14, 'reg_alpha': 0.05552669174004466, 'reg_lambda': 0.03610338183325372, 'subsample': 0.8812246089748421, 'colsample_bytree': 0.7474548822154962, 'colsample_bylevel':

In [ ]:
print(f"Best score: {study.best_trial.value}")
study.best_trial.params

Best score: 0.7509631471504659


{'colsample_bylevel': 0.834642539372038,
 'colsample_bynode': 0.5637465116572634,
 'colsample_bytree': 0.5277014454268698,
 'gamma': 1.3734018462451032,
 'grow_policy': 'depthwise',
 'learning_rate': 0.013059591966818461,
 'max_depth': 15,
 'min_child_weight': 7,
 'n_estimators': 756,
 'reg_alpha': 0.0179735395470121,
 'reg_lambda': 0.059126402132885794,
 'subsample': 0.8076140767507589}

### Evaluate Best Model

In [15]:
hyperparams = {
    'objective': 'multi:softmax',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'nthread': 6,
    'seed': 69,
    'colsample_bylevel': 0.834642539372038,
    'colsample_bynode': 0.5637465116572634,
    'colsample_bytree': 0.5277014454268698,
    'gamma': 1.3734018462451032,
    'grow_policy': 'depthwise',
    'learning_rate': 0.013059591966818461,
    'max_depth': 15,
    'min_child_weight': 7,
    'n_estimators': 756,
    'reg_alpha': 0.0179735395470121,
    'reg_lambda': 0.059126402132885794,
    'subsample': 0.8076140767507589
}

X_train, X_valid, y_train, y_valid = train_test_split(
    features_df, 
    labels_df.to_numpy().squeeze(), 
    stratify=labels_df.to_numpy().squeeze(),
    random_state=69)

X_train = preprocessor_xgboost.fit_transform(X_train, y_train)
X_valid = preprocessor_xgboost.transform(X_valid)

xgb_clf = XGBClassifier(
    **hyperparams, 
    early_stopping_rounds=50,
)

xgb_clf.fit(
    X_train, 
    y_train,  
    eval_set=[(X_valid, y_valid)],
    verbose=False,
)
results = f1_score(y_valid, xgb_clf.predict(X_valid), average='micro')
results

0.7509171002747465

## Submission

In [16]:
from os import PathLike
import pandas as pd


def create_submission(predictions, submission_formats_path: PathLike):
    submission_format = pd.read_csv(submission_formats_path, index_col=0)
    submission = pd.DataFrame(data=predictions, columns=submission_format.columns, index=submission_format.index)
    submission['damage_grade'] = submission['damage_grade'].astype(int)
    return submission

In [17]:
X_test = preprocessor_xgboost.transform(test_features_df)


submission = create_submission(xgb_clf.predict(X_test) + 1, submission_formats_path=SUBMISSION_FORMAT_PATH)
submission.to_csv(SUBMISSION_DIR / "xgb-DR-32-RU-16-Catboost-raw.csv")

## Ensemble

### Bagging

In [ ]:
K = 5

models = [None for _ in range(K)]
weights = np.zeros(K)

for i, (train_idx, valid_idx) in enumerate(StratifiedKFold(n_splits=K)):
  X_train, y_train = features_df.iloc[train_idx], labels_df.iloc[train_idx].to_numpy().squeeze()
  X_valid, y_valid = features_df.iloc[valid_idx], labels_df.iloc[valid_idx].to_numpy().squeeze()
  X_train = preprocessor_xgboost.fit_transform(X_train, y_train)
  X_valid = preprocessor_xgboost.transform(X_valid)
  models[i] = XGBClassifier(**hyperparams, early_stopping_rounds=50).fit(
    X_train, y_train,  
    eval_set=[(X_valid, y_valid)], verbose=False)
  
  weights[i] = f1_score(y_valid, xgb_clf.predict(X_valid), average='micro')

weights = weights / weights.sum()

In [ ]:
preprocessor_xgboost.fit(features_df, labels_df.to_numpy().squeeze())
X_test = preprocessor_xgboost.transform(test_features_df)

preds = np.zeros(X_test.shape[0])
### Predict
for i in range(K):
  preds += models[i].predict_proba(X_test) * weights[i]

predictions = preds.argmax(axis=1)

In [ ]:
submission = create_submission(predictions + 1, submission_formats_path=SUBMISSION_FORMAT_PATH)
submission.to_csv(SUBMISSION_DIR / "xgb-DR-32-RU-16-Target-bagged.csv")